# Import the necessary Packages

In [1]:
import bs4
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_cohere import CohereEmbeddings
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
import os
import json

# Setup & Keys

In [2]:
# Load the secret keys from the JSON file
def load_secret_keys(file_path):
    with open(file_path, 'r') as file:
        secret_keys = json.load(file)
    return secret_keys
keys =load_secret_keys('secrets.json')

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = keys['Langchain Smith']
os.environ["COHERE_API_KEY"] = keys['cohere']
os.environ["GROQ_API_KEY"] = keys['Groq']

# Load the pdf Documents

In [4]:
### Construct retriever ###
# Replace WebBaseLoader with PyPDFLoader to load PDF documents
#pdf_url = "https://arxiv.org/pdf/2005.11401"
pdf_url = "https://www.gov.nl.ca/ecc/files/env-protection-pesticides-business-manuals-applic-chapter7.pdf"
loader = PyPDFLoader('env-protection-pesticides-business-manuals-applic-chapter7.pdf')
docs = loader.load()

In [5]:
print(len(docs))
print(docs[0])

28
page_content='APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                   169 INTEGRATED PEST 
MANAGEMENT  
The goal of pest management is to manage pests effectively, economically, and 
safely. Insects, weeds, plant diseases, slugs, birds, and mammal pests can be managed using Integrated Pest Management (IPM). With IPM, you only need to reduce pest numbers below a damaging level.  It is not necessary to eliminate all 
pests.   The IPM approach was first developed for agricultural pests. Since the 1980s, it 
has been successfully applied to:   
 Landscape pests  
 Forestry pests  
 Structural pests  
 Home and garden pests  
 IPM helps decrease the need for chemical pesticides. This reduces costs and environmental risks.           Chapter 
7 
Learning Objectives  
Completing this chapter will help you to:  
 Define Integrated Pest Management  and describe the parts of an 
IPM program. 
 
 Know why correct identifi cation and knowledge of pest biolo

# Preprocessing

## Use RecursiveCharacterTextSplitter to chunk the document

In [6]:
# Use RecursiveCharacterTextSplitter to chunk the document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [7]:
print(len(splits))
#print(splits[0])

splits[5].dict()

57


{'id': None,
 'metadata': {'source': 'env-protection-pesticides-business-manuals-applic-chapter7.pdf',
  'page': 2},
 'page_content': 'cheaper and gives better results in the l ong run. Waiting until problems occur can \ncreate a reliance on treatments. Preventing pest problems protects plants with no \ntreatment cost. If prevention does not eliminate pests, their numbers are at least lower. This makes them easier to control.  Cultural or physical controls (described later) can be considered preventive. You might choose disease resistant plant speci es to avoid some disease problems. \nSanitation is a big part of prevention. An  example would be cleaning a warehouse \nto deny food and water to rodents.   \nIdentification \n Not all plant damage or decline is caused by pests. Crops, ornamentals, lawns, and \nother plants can be damaged by extreme heat, cold, or wind. They can also be damaged by physical injuries from equipment,  lack of nutrients, human activity, or \npet urine. Many ty

## Create a vector store using the split documents and Cohere embeddings

In [8]:
# Create a vector store using the split documents and Cohere embeddings
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=CohereEmbeddings()
                                   )
retriever = vectorstore.as_retriever()

In [9]:
type(retriever)
retriever.dict()

{'name': None,
 'tags': ['Chroma', 'CohereEmbeddings'],
 'metadata': None,
 'vectorstore': <langchain_chroma.vectorstores.Chroma at 0x74a6e616e810>,
 'search_type': 'similarity',
 'search_kwargs': {}}

In [10]:
#retriever.vectorstore.get_by_ids(ids='045f6867-07e5-49d1-99c4-52a9cfe029a0')
retriever.vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data', 'included'])

In [11]:
print(retriever.vectorstore.get()['ids'][-1])
print('-'*25)
print(retriever.vectorstore.get()['metadatas'][-1])
print('-'*25)
print(retriever.vectorstore.get()['documents'][-1])
print('-'*25)
print(retriever.vectorstore.get()['included'][-1])
print('-'*25)
#print(retriever.vectorstore.get()['embeddings'][-1])

fbf54729-50ac-44b2-af82-b5e67c5935ce
-------------------------
{'page': 1, 'source': 'env-protection-pesticides-business-manuals-applic-chapter7.pdf'}
-------------------------
biological, physical, mechanical, beha vioural, or chemical methods. The 
goal is to control pests with little impact on the environment.  
6. Evaluation : The effectiveness of pest management plans are considered.   
 Principles of Integrated Pest 
Management 
 Learning Objectives, cont’d.  
 
 Describe injury and action thresholds and know the difference 
between them. 
 
 Describe five categories of pest  treatments and give examples. 
 
 Know  the factors to consider when choosing pest treatments. 
 
 Know the importance of evaluati ng pest management results.
-------------------------
documents
-------------------------


# Load the LLM model

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

# ChatPromptTemplate

In [13]:
# Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for finding the exact or similar answer to questions using the provided context."
    "the question. If you don't know the answer, say that you don't know."
    " Keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# RAG Chain

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain.chains import create_retrieval_chain

rag_chain = create_retrieval_chain(retriever, question_answer_chain)
#rag_chain = create_retrieval_chain(splits, question_answer_chain)

In [16]:
response = rag_chain.invoke({"input": "Are aphids a pest?"})

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:According to the provided context, aphids are mentioned as a type of pest.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 4
Content: APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                  173  
 Understanding pest behavior can help in choosing the time and place of 
treatment. Some pests are only a...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 3
Content: APPLICATOR CORE 
172                                  Chapter 7: Integrated Pest Management To plan an IPM program, find out: 
 
 When pests will be present  
 What they eat  
 Where they hide  
 ...
--------------------------------------------------
Context 3:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 2
Content: pet urine. Many types of pest damage lo

In [17]:
# Define the question
question = {"input":  "What is integrated Pest Management"}

# Invoke the RAG chain
response = rag_chain.invoke(question)
# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:According to the provided context, Integrated Pest Management (IPM) is a decision-making process that helps to prevent pest problems. It is a program that considers all information and treatment methods to manage pests effectively, economically, and safely, with the goal of reducing pest numbers below a damaging level, rather than eliminating all pests.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 0
Content: APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                   169 INTEGRATED PEST 
MANAGEMENT  
The goal of pest management is to manage pests effectively, economically, and...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 1
Content: APPLICATOR CORE 
170                                  Chapter 7: Integrated Pest Management  
 
 
 
 
 
 
  
 
 IPM is a decisi

In [18]:
# Define the question
question = {"input":   "What is IPM program?"}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:An IPM (Integrated Pest Management) program is a decision-making process that helps to prevent pest problems by considering all available information and treatment methods to manage pests. This approach aims to be effective, affordable, and safe for the environment.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 11
Content:  Government and scientific texts 
 Pest management experts 
 Universities or colleges 
 Grower organizations 
 
Threshold information may not always be av ailable. It may be necessary to begin 
by...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 1
Content: APPLICATOR CORE 
170                                  Chapter 7: Integrated Pest Management  
 
 
 
 
 
 
  
 
 IPM is a decision-making process that he lps to prevent pest problems. With IPM 
program...
-----------

In [19]:
question = {"input":"What are the key elements of an IPM Program?"}

# Invoke the RAG chain
response = rag_chain.invoke(question)
# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:According to the provided context, the key elements of an IPM (Integrated Pest Management) program include:

1. Prevention: Organisms are kept from becoming problems by planning and managing ecosystems.
2. Identification: Pests and beneficial organisms are identified.
3. Monitoring: Pest and beneficial organism populations are watched, as well as pest damage, and the environment.
4. Injury and Action Decision: Injury and action thresholds are used to know when to treat pests.
5. Treatments: Treatments (or a combination) are used, including cultural, biological, physical, mechanical, behavioral, or chemical methods. The goal is to control pests with little impact on the environment.
6. Evaluation: Evaluation is done to refine and improve thresholds over time.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 1
Content: APPLICATOR CORE 
170                                  Chapter 7: Integrated Pest Ma

In [20]:
question = {"input":  "Is Monitoring,  Injury and Action Thresholds not the elements of an IPM Program?"}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:Yes, that's correct. Monitoring, Injury and Action Thresholds are indeed elements of an Integrated Pest Management (IPM) program.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 9
Content: Frequency of Monitoring 
 The way in which you monitor should depend on the pest and type of site. Monitoring for insects is often done weekly. It should be done during a time when 
damage is expected...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 11
Content:  Government and scientific texts 
 Pest management experts 
 Universities or colleges 
 Grower organizations 
 
Threshold information may not always be av ailable. It may be necessary to begin 
by...
--------------------------------------------------
Context 3:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf


In [21]:
question = {"input": "How to plan an IPM Program?"}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:According to the provided context, planning an IPM program involves the following steps:

1. Prevention: Plan and manage ecosystems to prevent organisms from becoming problems.
2. Identification: Identify pests and beneficial organisms.
3. Monitoring: Monitor pest and beneficial organisms' populations, pest damage, and the environment.
4. Injury and Action Decision: Determine injury and action thresholds to know when to treat pests.
5. Treatments: Use a combination of cultural, biological, physical, mechanical, behavioral, or chemical methods to control pests with minimal environmental impact.

Additionally, it is essential to:

* Evaluate the program for effectiveness and identify ways to prevent pest problems and improve the program.
* Communicate with all involved parties, including local pest experts, government employees, trade associations, and workers on farms or in businesses, to ensure everyone understands their role and the program's goals.
* Keep records of treatments

In [22]:
question = {"input":"""What are these in IPM program?
1. When pests will be present
2. What they eat
3. Where they hide
4. The stages of life that are easiest to control
5. What natural enemies exist
"""}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:These are the factors to consider when planning an IPM (Integrated Pest Management) program:

1. Timing: When pests will be present
2. Diet: What they eat
3. Habitat: Where they hide
4. Life cycle: The stages of life that are easiest to control
5. Biology: What natural enemies exist

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 3
Content: APPLICATOR CORE 
172                                  Chapter 7: Integrated Pest Management To plan an IPM program, find out: 
 
 When pests will be present  
 What they eat  
 Where they hide  
 ...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 1
Content: APPLICATOR CORE 
170                                  Chapter 7: Integrated Pest Management  
 
 
 
 
 
 
  
 
 IPM is a decision-making process that he lps to prevent pest problems. With IPM 
progra

In [23]:
question = {"input": """
What are the advantages of using an IPM program?
provide atleast 5 points
"""}
# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:According to the provided context, the advantages of using an IPM program include:

1. Long-term answers to pest problems
2. Protecting environmental and human health by reducing pesticide use
3. Reducing harm to beneficial organisms
4. Preventing creation of pesticide-resistant pests
5. Providing a way to manage pests when pesticides cannot be used

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 2
Content: APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                  171 Advantages of IPM include: 
 
 Long-term answers to pest problems 
 Protecting environmental and human hea...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 1
Content: APPLICATOR CORE 
170                                  Chapter 7: Integrated Pest Management  
 
 
 
 
 
 
  
 
 IPM is a decision-m

In [24]:
question = {"input": "Why is prevention key to an IPM program?"}
# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:According to the provided context, prevention is key to an IPM program because it keeps pests from causing problems, which avoids damage and saves money.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 24
Content: treatments. Treatment records can be  used to plan improvements and 
answer questions after treatment.  An IPM program must be evaluated fo r effectiveness. Ways to prevent 
pest problems and improve ...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 1
Content: APPLICATOR CORE 
170                                  Chapter 7: Integrated Pest Management  
 
 
 
 
 
 
  
 
 IPM is a decision-making process that he lps to prevent pest problems. With IPM 
program...
--------------------------------------------------
Context 3:
Type: Document
Source: env-protection-pesticides-business-manu

In [25]:
question = {"input": """ Choose the correct options from a,b,c,d & e

Monitoring is used in an IPM program to:
a. Discover if pests are present and in what numbers
b. Find pest damage or symptoms of disease
c. Determine if beneficial organisms are present
d. All of above
e. a and b only
"""}
# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:The correct answer is d. All of above.

According to the provided context, monitoring in an IPM program involves:

* Providing information needed to make sound decisions on pest management
* Comes from visual inspections or counting and measuring methods
* Regular counts or measurements provide numbers that can be compared over time, regardless of who does the monitoring
* Enough random samples must be taken to get a good estimate of the pest population

This suggests that monitoring is used to discover if pests are present and in what numbers (a), find pest damage or symptoms of disease (b), and determine if beneficial organisms are present (c). Therefore, the correct answer is d. All of above.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 11
Content:  Government and scientific texts 
 Pest management experts 
 Universities or colleges 
 Grower organizations 
 
Threshold information may not 

In [26]:
question = {"input": """Explain the difference between injury threshold and action threshold."""}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:According to the provided context, the injury threshold is the level at which pest numbers are high enough to cause unacceptable injury or damage. It is the maximum number of pests that can be tolerated.

On the other hand, the action threshold is the point at which treatment should take place to prevent the pest population from reaching the injury threshold. The action threshold may differ depending on the type of treatment and how it works.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 11
Content: APPLICATOR CORE 
180                                  Chapter 7: Integrated Pest Management Injury and action thresholds have been  well studied and established for some 
crops. These take the cost of...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 10
Content: APPLICATOR CORE 
Chapter 7: Integr

In [27]:
question = {"input":"""List five treatments used for pest control. Give one example of each."""}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER:{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER:Based on the provided context, here are five categories of pest treatments with one example each:

1. Cultural controls: These disrupt pest life cycles or make the environment less suited for survival.
Example: Crop rotation to reduce pest populations.

2. Mechanical and physical controls: These use equipment or devices, barriers, or temperatures to reduce pest populations.
Example: Using row covers to prevent pests from reaching crops.

3. Biological controls: These are living natural enemies of a pest.
Example: Introducing ladybugs to reduce aphid populations.

4. Behavioural controls: These use a pest's natural behavior to suppress the population.
Example: Releasing pheromones to confuse male insects and disrupt mating.

5. Chemical controls: These include most pesticides and are designed to kill, control, repel, attract, or manage pests.
Example: Using insecticides to kill pests that have infested a crop.

*************************
Context 1:
Type: Document
Source: env-prote

In [28]:
question = {"input": """Why is communication important in an IPM program?"""}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER :{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER :According to the provided context, communication is important in an IPM program because it helps to:

* Gather local experience and information
* Ensure everyone understands their role
* Pass along IPM information to customers and promote the benefits of the program
* Address customer concerns and promote customer satisfaction
* Plan improvements and answer questions after treatment
* Identify ways to prevent pest problems and improve the program

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 22
Content: APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                  191  
Communication is important when devel oping and putting an IPM program in 
place. A large amount of inform...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 22
Content: It is important to pass along

In [29]:
question = {"input": """A monitoring program should take enough samples to get the most
accurate estimate of the pest population.
True or False?"""}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER :{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER :True. According to the context, "The greater the number of samples counted, the more likely it is that the results will give a good estimate."

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 8
Content: APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                  177 SAMPLE SIZE 
 
The greater the number of samples counted, the more likely it is that the results 
will give ...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 7
Content:  Compare records between sites or dates 
 Establish injury levels and action thresholds 
 Evaluate the effect of treatments on pest populations 
  
Sampling Theory 
 
Counting methods often involve...
--------------------------------------------------
Context 3:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-

In [30]:
question = {"input":  """A monitoring program should take enough samples to get the most
accurate estimate of the pest population.
True or False?"""}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER :{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER :True.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 8
Content: APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                  177 SAMPLE SIZE 
 
The greater the number of samples counted, the more likely it is that the results 
will give ...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 7
Content:  Compare records between sites or dates 
 Establish injury levels and action thresholds 
 Evaluate the effect of treatments on pest populations 
  
Sampling Theory 
 
Counting methods often involve...
--------------------------------------------------
Context 3:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 23
Content: treatment. You must know what the problem is before choosing 
methods to monitor and treat it.  
 
Monitor

In [31]:
question = {"input": """Visual inspections include counting the number of pests on plants.
True or False?"""}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER :{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER :False. According to the text, visual inspections are "Close and careful examinations to see what pests are present" and "Done when and where they are needed", but it does not include counting the number of pests on plants. Counting and measuring methods are a separate category.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 6
Content:  Check for the presence of pests or pest damage 
 
 Check for the presence or absence of beneficial organisms. 
 
 Find large problem sites (e.g., weed y patches or infected crop areas). 
 
 Find ...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 6
Content: APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                  175 There are many types of monitoring m ethods and tools. Methods include visual 
inspections and counting and ...

In [32]:
question = {"input": """In an IPM program, only one treatment is used for a given pest problem.
True or False?
"""}

# Invoke the RAG chain
response = rag_chain.invoke(question)

# Print the main answer
print(f"ANSWER :{response['answer']}\n")

# Print each context briefly
print('*' * 25)
for idx, ctx in enumerate(response['context']):
    print(f"Context {idx + 1}:")
    print(f"Type: {ctx.type}")  # Accessing type using the attribute
    print(f"Source: {ctx.metadata['source']}")  # Accessing metadata using subscript notation
    print(f"Page: {ctx.metadata['page']}")  # Accessing metadata using subscript notation
    # Print a snippet of the content
    content_snippet = ctx.page_content[:200] + ('...' if len(ctx.page_content) > 200 else '')
    print(f"Content: {content_snippet}")
    print('-' * 50)

ANSWER :False.

*************************
Context 1:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 26
Content: APPLICATOR CORE 
Chapter 7: Integrated Pest Management                                  195 5. List five treatments used for pest control. Give one example of each. 
 
  
     
     6. Why is communic...
--------------------------------------------------
Context 2:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 11
Content:  Government and scientific texts 
 Pest management experts 
 Universities or colleges 
 Grower organizations 
 
Threshold information may not always be av ailable. It may be necessary to begin 
by...
--------------------------------------------------
Context 3:
Type: Document
Source: env-protection-pesticides-business-manuals-applic-chapter7.pdf
Page: 23
Content: APPLICATOR CORE 
192                                  Chapter 7: Integrated Pest Management   
        

# RAG Chain2

In [33]:
system_prompt = (
    "You are an assistant for finding the exact or similar answer to questions using the provided context."
    "the question. If you don't know the answer, say that you don't know."
    " Keep the answer concise."
    "\n\n"
    "{context}"
)

In [34]:
prompt_template = """
<|start_header_id|>user<|end_header_id|>
You are an assistant for finding the exact or similar answer to questions using the provided context.
The question. If you haven't found the context, I haven't found any result in the provided documents.
Keep the context concise.
Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [35]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [36]:
from langchain.chains import RetrievalQA

rag_chain2 = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": prompt})

In [37]:
# Define the question
question = "Are aphids a pest?"

# Invoke the QA chain
response = rag_chain2.invoke(question)
print(response.keys())

dict_keys(['query', 'result'])


In [38]:
# Define the question
question = "What is integrated Pest Management"

# Invoke the QA chain
result = rag_chain2.invoke(question)["result"]
print(result)

Based on the provided context, the answer to the question "What is Integrated Pest Management?" is:

Integrated Pest Management (IPM) is a decision-making process that helps to prevent pest problems by considering all available information and treatment methods. It is a holistic approach to managing pests, aiming to reduce pest numbers below a damaging level, not necessarily eliminating all pests. IPM programs include elements such as prevention, identification, monitoring, injury and action decision, treatments, and evaluation. The goal is to control pests effectively, affordably, and safely for the environment.


In [39]:
question = "What is IPM program?"

# Invoke the QA chain
result = rag_chain2.invoke(question)["result"]
print(result)

Based on the provided context, the IPM program refers to Integrated Pest Management, which is a decision-making process that helps to prevent pest problems. It considers all available information and treatment methods to manage pests in an effective, affordable, and safe manner for the environment.

An IPM program typically includes the following elements:

1. Prevention: planning and managing ecosystems to prevent organisms from becoming problems
2. Identification: identifying pests and beneficial organisms
3. Monitoring: tracking pest and beneficial organism populations, pest damage, and the environment
4. Injury and Action Decision: using injury and action thresholds to determine when to treat pests
5. Treatments: using a combination of cultural, biological, physical, mechanical, behavioral, or chemical methods to control pests

The goal of IPM is to manage pests effectively, economically, and safely, and it has been successfully applied to various sectors, including agriculture, la

In [40]:
question = "What are the key elements of an IPM Program? brief each elements pointwise."

print(rag_chain2.invoke(question)["result"])

Based on the provided context, the key elements of an IPM (Integrated Pest Management) Program are:

1. **Prevention**:
	* Plan and manage ecosystems to prevent organisms from becoming problems.
2. **Identification**:
	* Identify pests and beneficial organisms.
3. **Monitoring**:
	* Watch pest and beneficial organism populations, pest damage, and the environment.
4. **Injury and Action Decision**:
	* Use injury and action thresholds to determine when to treat pests.
5. **Treatments**:
	* Use cultural, biological, physical, mechanical, behavioral, or chemical methods to control pests with minimal environmental impact.

These elements are critical to an effective, affordable, and safe IPM program.


In [41]:
question = "Is Monitoring,  Injury and Action Thresholds not the elements of an IPM Program?"

print(rag_chain2.invoke(question)["result"])

Based on the provided context, it appears that the elements of an Integrated Pest Management (IPM) Program are:

1. Monitoring
2. Injury Threshold
3. Action Threshold

These elements are mentioned in the context as follows:

* Monitoring is discussed as a crucial step in IPM, involving regular counts or measurements to provide information for sound decisions on pest management.
* Injury Threshold is mentioned as the level at which pest numbers are high enough to cause unacceptable injury or damage, and the maximum number of pests that can be tolerated.
* Action Threshold is mentioned as the point at which treatment should take place to prevent the pest population from reaching the injury threshold.

These elements are key components of an IPM program, which aims to manage pests in a sustainable and environmentally friendly manner.


In [42]:
question = "How to plan an IPM Program? Provide stepwise guide"

print(rag_chain2.invoke(question)["result"])

Based on the provided context, I have found a stepwise guide on how to plan an IPM (Integrated Pest Management) program:

**Step 1: Prevention**

* Plan and manage ecosystems to prevent pest problems

**Step 2: Identification**

* Identify pests and beneficial organisms

**Step 3: Monitoring**

* Monitor pest and beneficial organism populations, as well as pest damage and the environment

**Step 4: Injury and Action Decision**

* Determine injury thresholds to know when to treat pests

**Step 5: Treatments**

* Select the best treatments, considering all available information and using a combination of methods (cultural, biological, physical, mechanical, behavioral, or chemical)

**Additional Steps:**

* Evaluate the IPM program annually to refine and improve thresholds
* Keep good records and use treatment records to plan improvements and answer questions after treatment
* Communicate effectively with all parties involved, including local pest experts, government employees, trade asso

In [43]:
question = """What are these in IPM program?
1. When pests will be present
2. What they eat
3. Where they hide
4. The stages of life that are easiest to control
5. What natural enemies exist
"""
print(rag_chain2.invoke(question)["result"])

Based on the provided context, I was able to find the answer to your question:

The 5 items in an IPM program are:

1. When pests will be present
2. What they eat
3. Where they hide
4. The stages of life that are easiest to control
5. What natural enemies exist

These items are mentioned in the context as the information to find out to plan an IPM program.


In [44]:
question = """
What are the advantages of using an IPM program?
provide atleast 5 points
"""
print(rag_chain2.invoke(question)["result"])

Based on the provided context, here are the advantages of using an IPM program:

1. Long-term answers to pest problems: IPM provides a sustainable solution to pest management, ensuring that the problem is resolved without creating new issues.
2. Protecting environmental and human health by reducing pesticide use: IPM reduces the reliance on chemical pesticides, minimizing harm to the environment and human health.
3. Reducing harm to beneficial organisms: IPM considers the impact on beneficial organisms, such as bees and butterflies, and aims to minimize harm to these important species.
4. Preventing creation of pesticide-resistant pests: By using a combination of control methods, IPM reduces the likelihood of pests developing resistance to pesticides.
5. Providing a way to manage pests when pesticides cannot be used: IPM offers alternative control methods when pesticides are not effective or cannot be used, ensuring that pest management is still possible.

These advantages highlight th

In [45]:
question = """
Why is prevention key to an IPM program?
"""
print(rag_chain2.invoke(question)["result"])

Based on the provided context, I found the answer to the question:

"Why is prevention key to an IPM program?"

Prevention is key to an IPM program because it keeps pests from causing problems, avoiding damage and saving money. Prevention is a key step in IPM, and it's often cheaper and gives better results in the long run.


In [46]:
question = """ Choose the correct options from a,b,c,d & e

Monitoring is used in an IPM program to:
a. Discover if pests are present and in what numbers
b. Find pest damage or symptoms of disease
c. Determine if beneficial organisms are present
d. All of above
e. a and b only
"""

print(rag_chain2.invoke(question)["result"])

Based on the provided context, the correct options are:

d. All of above

Monitoring is used in an IPM program to:

* Discover if pests are present and in what numbers (Option a)
* Find pest damage or symptoms of disease (Option b)
* Determine if beneficial organisms are present (Option c)

All of these options are mentioned in the context as part of the monitoring process in an IPM program.


In [47]:
question = """Explain the difference between injury threshold and action threshold."""

print(rag_chain2.invoke(question)["result"])

Based on the provided context, I found the answers to your question.

**Injury Threshold:**
The injury threshold is the level at which pest numbers are high enough to cause unacceptable injury or damage. It is the maximum number of pests that can be tolerated.

**Action Threshold:**
The action threshold is the point at which treatment should take place to prevent the pest population from reaching the injury threshold. It depends on the type of treatment, with faster-acting treatments often requiring treatment at a higher population level and slower-acting treatments requiring treatment earlier.

Please let me know if you have any further questions or if there's anything else I can help you with!


In [48]:
question = """List five treatments used for pest control. Give one example of each."""

print(rag_chain2.invoke(question)["result"])     

Based on the provided context, here are five treatments used for pest control, along with one example of each:

1. **Biological Control**: Releasing insect pheromones to confuse male insects and disrupt mating.
2. **Physical Control**: Using traps or barriers to reduce pest populations.
3. **Mechanical Control**: Releasing sterile male insects to prevent pests from reproducing (e.g., codling moth).
4. **Behavioural Control**: Releasing insect pheromones to send signals to others over a large area, confusing male insects and disrupting mating.
5. **Chemical Control**: Using pesticides, such as insecticides, to kill, control, or repel pests.

Note that these examples are from the provided context and may not be an exhaustive list of treatments used for pest control.


In [49]:
question = """Why is communication important in an IPM program?"""

print(rag_chain2.invoke(question)["result"])     

Based on the provided context, I found the answer to the question "Why is communication important in an IPM program?"

According to the text, communication is important in an IPM program because:

* A large amount of information must be taken into account and applied to solve a pest problem.
* Local pest experts, government employees, trade associations, and other IPM practitioners gather local experience and information, which must be communicated.
* Workers on farms or in businesses involved in pest management must communicate with each other.
* Details of the IPM program and its goals must be made clear to workers.
* There should be a method to relay needs and observations to employers or supervisors.
* Everyone should clearly understand their role for the program to succeed.
* IPM information must be passed along to customers and the general public, and the benefits of the program must be promoted to them.
* Communication helps to address customer concerns, promote customer satisfa

In [50]:
question = """A monitoring program should take enough samples to get the most
accurate estimate of the pest population.
True or False?"""

print(rag_chain2.invoke(question)["result"])     

Based on the provided context, the answer to the question is:

**True**

The context states that "The greater the number of samples counted, the more likely it is that the results will give a good estimate." It also mentions that taking enough samples is necessary to get a good estimate of the pest population, and that 10 to 50 samples are often required. Additionally, the text explains that comparing the averages of 10 and 40 samples can help determine if taking more samples will provide a more accurate estimate.


In [51]:
question = """A monitoring program should take enough samples to get the most
accurate estimate of the pest population.
True or False?"""

print(rag_chain2.invoke(question)["result"])     

Based on the provided context, the answer to the question is:

**True**

According to the text, "The greater the number of samples counted, the more likely it is that the results will give a good estimate. Ten to fifty samples are often required." This suggests that taking enough samples is necessary to get an accurate estimate of the pest population.


In [52]:
question = """Visual inspections include counting the number of pests on plants.
True or False?"""

print(rag_chain2.invoke(question)["result"])     

Based on the provided context, the answer to the question is:

True

The context states that visual inspections include "Close and careful examinations to see what pests are present" and "Counting the number of pests on plants" is mentioned in the context of Counting and Measuring Methods.


In [53]:
question = """In an IPM program, only one treatment is used for a given pest problem.
True or False?
"""
print(rag_chain2.invoke(question)["result"])

Based on the provided context, the answer to the question is:

**False**

The context mentions that in an IPM program, "Two or more treatments are often used together" and that "Treatments should only be used if experience and monitoring information shows that pest numbers are likely to increase to the injury threshold." This suggests that multiple treatments may be used for a given pest problem, rather than just one treatment being used.
